In [1]:
%matplotlib inline

In [1]:
import cv2
import os
import math
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten,MaxPooling2D,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from tensorflow.keras.optimizers import Adam,SGD
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.linear_model import LogisticRegression
from mostCommon import most_common,patient_score

In [2]:
IMAGE_WIDTH=224
IMAGE_HEIGHT=224

In [3]:
#Load all patients 
my_data = pd.read_csv('../data/Fifteen_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Fifteen_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2) 
images_data = []
models = []

df=my_data
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img=  cv2.equalizeHist(img)
    img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    img_2d = img.reshape(IMAGE_WIDTH ,IMAGE_HEIGHT,1)
    images_data.append(img_2d)

X=np.asarray(images_data)
y = np.asarray(my_data['Label'])
print(Counter(y))

Counter({'regular': 840, 'pneumonia': 540, 'covid': 450})


In [4]:
testPatient_indexes= [1,2,12,14,15,22,
                      33,34,37,42,43,52,53,
                      92,95,96,99,101,102,104,107,110,115,118]


In [5]:
#getting frame indexes of each patient
frames_index=[]
for i in testPatient_indexes:
  j=i*15
  z=j+15
  while j<z:
    frames_index.append(j)
    j+=1

# Adding the frames to X_test
X_test=np.take(X,frames_index,axis=0)
y_test=np.take(y,frames_index,axis=0)
# Removing the frames from X_train
X_train=np.delete(X,frames_index,axis=0)
y_train=np.delete(y,frames_index,axis=0)

# Scale the pixels
X_train=X_train/255.0
X_test=X_test/255.0

In [6]:
#Applying SMOTE
oversample = SMOTE()
X_train=np.resize(X_train,(X_train.shape[0],50176))
X_train,y_train=oversample.fit_resample(X_train, y_train)
counter=Counter(y_train)
print(counter)

Counter({'covid': 675, 'pneumonia': 675, 'regular': 675})


In [7]:
print('X_train: '+str(X_train.shape))
print('y_train: '+str(y_train.shape))
print('X_test: '+ str(X_test.shape))
print('y_test: '+ str(y_test.shape))

X_train: (2025, 50176)
y_train: (2025,)
X_test: (360, 224, 224, 1)
y_test: (360,)


In [8]:
model=LogisticRegression(random_state=42,solver='lbfgs',max_iter=10000)
model.fit(X_train,y_train)

LogisticRegression(max_iter=10000, random_state=42)

In [9]:
X_test=np.resize(X_test,(X_test.shape[0],50176))
y_pred=model.predict(X_test)

In [10]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred, digits=3))
patient_score(y_test,y_pred)

[[ 54  12  24]
 [  0  74  31]
 [  0  19 146]]
              precision    recall  f1-score   support

       covid      1.000     0.600     0.750        90
   pneumonia      0.705     0.705     0.705       105
     regular      0.726     0.885     0.798       165

    accuracy                          0.761       360
   macro avg      0.810     0.730     0.751       360
weighted avg      0.788     0.761     0.759       360

[[ 4  1  1]
 [ 0  6  1]
 [ 0  1 10]]
              precision    recall  f1-score   support

       covid      1.000     0.667     0.800         6
   pneumonia      0.750     0.857     0.800         7
     regular      0.833     0.909     0.870        11

    accuracy                          0.833        24
   macro avg      0.861     0.811     0.823        24
weighted avg      0.851     0.833     0.832        24



In [11]:
>>> from sklearn.model_selection import cross_val_score

scores=cross_val_score(model, X_train, y_train, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.86 accuracy with a standard deviation of 0.08


In [12]:


IMAGE_WIDTH=224
IMAGE_HEIGHT=224
#Load all patients 
my_data = pd.read_csv('../data/Fifteen_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Fifteen_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2) 
images_data = []
models = []

df=my_data
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img=  cv2.equalizeHist(img)
    img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    img_2d = img.reshape(IMAGE_WIDTH ,IMAGE_HEIGHT,1)
    images_data.append(img_2d)

X=np.asarray(images_data)
y = np.asarray(my_data['Label'])
print(Counter(y))

testPatient_indexes= [1,2,12,14,15,22,
                      33,34,37,42,43,52,53,
                      92,95,96,99,101,102,104,107,110,115,118]
#getting frame indexes of each patient
frames_index=[]
for i in testPatient_indexes:
  j=i*15
  z=j+15
  while j<z:
    frames_index.append(j)
    j+=1

# Adding the frames to X_test
X_test=np.take(X,frames_index,axis=0)
y_test=np.take(y,frames_index,axis=0)
# Removing the frames from X_train
X_train=np.delete(X,frames_index,axis=0)
y_train=np.delete(y,frames_index,axis=0)

# Scale the pixels
X_train=X_train/255.0
X_test=X_test/255.0
X_train=np.resize(X_train,(X_train.shape[0],50176))
print('X_train: '+str(X_train.shape))
print('y_train: '+str(y_train.shape))
print('X_test: '+ str(X_test.shape))
print('y_test: '+ str(y_test.shape))
model=LogisticRegression(random_state=42,solver='lbfgs',max_iter=10000)
model.fit(X_train,y_train)
X_test=np.resize(X_test,(X_test.shape[0],50176))
y_pred=model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred, digits=3))
patient_score(y_test,y_pred)
from sklearn.model_selection import cross_val_score

scores=cross_val_score(model, X_train, y_train, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Counter({'regular': 840, 'pneumonia': 540, 'covid': 450})
X_train: (1470, 50176)
y_train: (1470,)
X_test: (360, 224, 224, 1)
y_test: (360,)
[[ 54  12  24]
 [  0  70  35]
 [  0  18 147]]
              precision    recall  f1-score   support

       covid      1.000     0.600     0.750        90
   pneumonia      0.700     0.667     0.683       105
     regular      0.714     0.891     0.792       165

    accuracy                          0.753       360
   macro avg      0.805     0.719     0.742       360
weighted avg      0.781     0.753     0.750       360

[[ 4  1  1]
 [ 0  6  1]
 [ 0  1 10]]
              precision    recall  f1-score   support

       covid      1.000     0.667     0.800         6
   pneumonia      0.750     0.857     0.800         7
     regular      0.833     0.909     0.870        11

    accuracy                          0.833        24
   macro avg      0.861     0.811     0.823        24
weighted avg      0.851     0.833     0.832        24

0.63 accuracy w

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
#Load all patients 
my_data = pd.read_csv('../data/Fifteen_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Fifteen_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2) 
images_data = []
models = []

df=my_data
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img=  cv2.equalizeHist(img)
    img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    img_1d = img.reshape(224 * 224)
    images_data.append(img_1d)

X=np.asarray(images_data)
y = np.asarray(my_data['Label'])
print(Counter(y))
scaler = StandardScaler()
# Fit on training set only
scaler.fit(X)
# Apply transform on both training and test set

X = scaler.transform(X)
pca = PCA(.95)
pca.fit(X)
X = pca.transform(X)

testPatient_indexes= [1,2,12,14,15,22,
                      33,34,37,42,43,52,53,
                      92,95,96,99,101,102,104,107,110,115,118]
#getting frame indexes of each patient
frames_index=[]
for i in testPatient_indexes:
  j=i*15
  z=j+15
  while j<z:
    frames_index.append(j)
    j+=1

# Adding the frames to X_test
X_test=np.take(X,frames_index,axis=0)
y_test=np.take(y,frames_index,axis=0)
# Removing the frames from X_train
X_train=np.delete(X,frames_index,axis=0)
y_train=np.delete(y,frames_index,axis=0)


print('X_train: '+str(X_train.shape))
print('y_train: '+str(y_train.shape))
print('X_test: '+ str(X_test.shape))
print('y_test: '+ str(y_test.shape))
model=LogisticRegression(random_state=42,solver='lbfgs',max_iter=10000)
model.fit(X_train,y_train)

y_pred=model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred, digits=3))
patient_score(y_test,y_pred)
from sklearn.model_selection import cross_val_score

scores=cross_val_score(model, X_train, y_train, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Counter({'regular': 840, 'pneumonia': 540, 'covid': 450})
X_train: (1470, 116)
y_train: (1470,)
X_test: (360, 116)
y_test: (360,)
[[ 53  25  12]
 [  0  39  66]
 [  0  25 140]]
              precision    recall  f1-score   support

       covid      1.000     0.589     0.741        90
   pneumonia      0.438     0.371     0.402       105
     regular      0.642     0.848     0.731       165

    accuracy                          0.644       360
   macro avg      0.693     0.603     0.625       360
weighted avg      0.672     0.644     0.638       360

[[ 3  2  1]
 [ 0  2  5]
 [ 0  1 10]]
              precision    recall  f1-score   support

       covid      1.000     0.500     0.667         6
   pneumonia      0.400     0.286     0.333         7
     regular      0.625     0.909     0.741        11

    accuracy                          0.625        24
   macro avg      0.675     0.565     0.580        24
weighted avg      0.653     0.625     0.603        24

0.61 accuracy with a stan